<a href="https://colab.research.google.com/github/WilliamAshbee/splineexample/blob/main/LSTM_mcfixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

With progressive growing of the sequence, the loss drops more stably. 

https://r2rt.com/non-zero-initial-states-for-recurrent-neural-networks.html

In [1]:
import torch
import numpy as np
import pylab as plt
from skimage import filters
import math

global numpoints
numpoints = 1000
side = 32

rows = torch.zeros(32,32)
columns = torch.zeros(32,32)


for i in range(32):
    columns[:,i] = i
    rows[i,:] = i


def donut_matrix(length = 10):
    radiusMax = side /3
    w = 1
    sigmas = [None, 1]
    
    canvas = torch.zeros((length,side, side))
    r0 = torch.tensor(np.random.uniform(side/4, side/3, length))

    radii = torch.zeros((length,numpoints))
    radii[:, :] = r0.unsqueeze(1)
    
    ind = [x for x in range(numpoints)]

    theta = torch.FloatTensor(ind)
    theta *= math.pi*2.0/(float)(numpoints)
    
    for i in range(1,length):
        a = np.random.uniform(1.0,3.0)*torch.sin(np.random.uniform(20.0)*theta+np.random.uniform(1000.0))
        #a = 4.0*torch.sin(10.0*theta)
        #print(a.shape,torch.max(a))
        radii[i,:] += a
        #print(radii.shape, torch.max(radii))
    
    assert torch.min(radii)>0
    #print(radii.max(axis = 0)[0].shape)
    rmaxs = radii.max(axis = 1)[0]
    pmins = rmaxs+1.0
    pmaxs = side-rmaxs-1.0
    x0 = np.random.uniform(pmins,pmaxs)
    y0 = np.random.uniform(pmins,pmaxs)
    x0[:]=side/2
    y0[:]=side/2
    x0 = torch.tensor(x0)
    y0 = torch.tensor(y0)
    
    x0 = x0.unsqueeze(1)
    y0 = y0.unsqueeze(1)
    #radii = torch.from_numpy(radii)
    xrfactors = torch.cos(theta).unsqueeze(0)
    yrfactors = torch.sin(theta).unsqueeze(0)
    
    print('x0_y0_r_xrf_yrf',x0.shape,y0.shape,radii.shape,xrfactors.shape,yrfactors.shape)

    x = (x0+(xrfactors*radii))
    y = (y0+(yrfactors*radii))
    assert x.shape == (length,numpoints)
    assert y.shape == (length,numpoints)
    assert torch.sum(x[x>(side-1)])==0 
    assert torch.sum(x[x<0])==0 
    assert torch.sum(y[y>(side-1)])==0 
    assert torch.sum(y[y<0])==0 
    
    points = torch.zeros(length,numpoints,2)
    for l in range(length):
        canvas[l,y[l,:].type(torch.LongTensor),x[l,:].type(torch.LongTensor)]=1.0
        points[l,:,0] = x[l,:]#modified for lstm discriminator
        points[l,:,1] = y[l,:]#modified for lstm discriminator 
    
    
    return {
        'canvas': canvas, 
        'points':points.type(torch.FloatTensor)}

def plot_all_model( sample = None, label=None):
    X = label[:,0]
    Y = label[:,1]
    img = sample[:,:].squeeze().cpu().numpy()
    plt.imshow(img, cmap=plt.cm.gray_r)
    predres = 1000

    s = [.001 for x in range(predres)]
    
    assert len(s) == predres
    c = ['red' for x in range(predres)]
    assert len(c) == predres
    plt.scatter(X.cpu().numpy(),Y.cpu().numpy(),s = s,c = c)


def getXYs(model=None,loader_disp = None):

    outputs = torch.zeros(100,1000,2).cuda()
    with torch.no_grad():
      h0 = None
      c0 = None
      X = []
      Y = []
      for samples,labels in loader_disp:
        for i in range(23):
          optimizer.zero_grad()
          if i == 0:
            out, h0, c0,o,output = model(samples.cuda(),labels[:,0,:2].cuda(),h0,c0,ind = i)
            outputs[:,i*25:(i+1)*25,:] = out.detach().clone()
          else:
            out, h0, c0,o,output = model(samples.cuda(),labels[:,0,:2].cuda(),h0,c0,ind = i,o = o,output=output)
            outputs[:,i*25:(i+1)*25,:] = out.detach().clone()

        return samples, outputs

          #X.extend(out.detach()[0,:,0].tolist())
          #Y.extend(out.detach()[0,:,1].tolist())

          
        

def plot_all( sample = None, labels = None):
    img = sample[:,:].squeeze().cpu().numpy()
    plt.imshow(img, cmap=plt.cm.gray_r)
    X = labels[:,0]
    Y = labels[:,1]
    s = [.001 for x in range(numpoints)]
    c = ['red' for x in range(numpoints)]
    plt.scatter(X.cpu().numpy(),Y.cpu().numpy(),s = s,c = c)
    
class DonutDataset(torch.utils.data.Dataset):
    """Donut dataset."""
    def __init__(self, length = 10):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.length = length
        self.values = donut_matrix(length)
        assert self.values['canvas'].shape[0] == self.length
        assert self.values['points'].shape[0] == self.length
        
        count = 0
        for i in range(self.length):
          a = self[i]
          c = a[0]
          for el in a[1]:
            #print(c[(int)(el[1]),(int)(el[0])].item())
            #assert c[(int)(el[1]),(int)(el[0])].item() == 1
            y,x = (int)(el[1]),(int)(el[0])
            if x < side-2 and x > 2 and y < side-2 and y > 2: 
              if c[y,x] != 1 and \
                c[y+1,x] != 1 and c[y+1,-1+x] != 1 and c[y+1,1+x] != 1 and \
                c[y-1,x] != 1 and c[y,-1+x] != 1 and c[y,1+x] != 1:
                count+=1
        assert count ==0
    def __len__(self):
        return self.length


    def __getitem__(self, idx):
        canvas = self.values["canvas"]
        
        canvas = canvas[idx,:,:]
        assert canvas.shape == (side,side)
        
        points = self.values["points"]
        points = points[idx,:]
        #points = points.unsqueeze(1)
        #z = torch.zeros(numpoints,1)
        #print(z.shape)
        #points = torch.cat([points,z], dim = 1)
        
        #print('points', points.shape)
        return canvas, points
    
    @staticmethod
    def displayCanvas(title,dataset, model):
        #model.setBatchSize(batch_size = 1)
        if model != None:
          model = model.eval()
          loader_disp = data.DataLoader(
            dataset, 
            batch_size=100,
            num_workers=2)
          samples, outputs = getXYs(model=model,loader_disp = loader_disp)
          for i in range(100):
            plt.subplot(10,10,i+1)
            plot_all_model(samples[i,:,:],outputs[i,:,:])
            plt.axis('off')

        else:
          for i in range(100):
            sample, labels = dataset[i]
            plt.subplot(10,10,i+1)
            plot_all(sample = sample,labels = labels)
            plt.axis('off')

        plt.savefig(title,dpi=450)
        plt.clf()
        plt.cla()
        plt.close()#should free memory
        print('open file: ',title)
dataset = DonutDataset(length = 100)

DonutDataset.displayCanvas('donut.png',dataset, model = None)


x0_y0_r_xrf_yrf torch.Size([100, 1]) torch.Size([100, 1]) torch.Size([100, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])
open file:  donut.png


In [2]:

from torch.utils import data
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

mini_batch = 100
dataset = DonutDataset(length = 100*20)
loader_train = data.DataLoader(
    dataset, 
    batch_size=mini_batch,
    sampler=RandomSampler(data_source=dataset),
    num_workers=2)


x0_y0_r_xrf_yrf torch.Size([2000, 1]) torch.Size([2000, 1]) torch.Size([2000, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


In [3]:
import torch
import torch.nn as nn
from torch.hub import load_state_dict_from_url

class DecoderRNN(nn.Module):
    def __init__(self):
        super(DecoderRNN, self).__init__()
        self.track = 0
        # define the properties
        self.embed_size = 2
        self.hidden_size = 512
        self.num_layers = 2
        self.seq_len = 25
        self.full_len = 1000
        self.lm_len = int(self.full_len/self.seq_len)
        
        self.longtermMem_lstm = None
        self.longtermMem_fc = None
        self.longtermMemP = None
        self.lstm_cell = nn.LSTM(self.hidden_size, self.hidden_size,self.num_layers)

        self.inject_image_lstm =  nn.Sequential(
            nn.Linear(2+1024+3*self.hidden_size,self.hidden_size),
            #nn.Dropout(),
            nn.Tanh()
        )

        
        
        self.fc_out = nn.Sequential(
          nn.Linear(2+1024+3*self.hidden_size,2),
        #    nn.ReLU(),
        #    nn.Dropout(),
        #    nn.Linear(10000,2),
            nn.Sigmoid()
        )
        

        self.alpha_lstm = nn.Sequential(
            nn.Linear(self.hidden_size,self.seq_len),
            nn.Softmax()
        )
        self.alpha_fc = nn.Sequential(
            nn.Linear(self.hidden_size,self.seq_len),
            nn.Softmax()
        )

        self.alpha_long_lstm = nn.Sequential(
            nn.Linear(self.hidden_size,self.lm_len),
            nn.Softmax()
        )
        
        

        self.embs = None #= torch.empty((self.batch_size, 1000, self.hidden_size)).cuda()
        
        
        

            
        
    def forward(self, features,p0,h0 = None, c0= None, ind = 0, o = None,output = None):
        assert features != None
        #assert captions != None
        #print('featursescaptions2',features.shape,captions.shape)
        #assert features.shape[0]==captions.shape[0]
        assert len(features.shape) == 3
        self.track +=1
        batch_size = features.shape[0]
        

        #img_emb = torch.empty((batch_size, captions.size(1), self.hidden_size)).cuda()
        
        p0 = torch.flatten(p0[:,:2],start_dim=1)
        

        if o == None:
          out = p0
          self.longtermMem_lstm = torch.zeros(batch_size,self.lm_len,self.hidden_size).cuda()
          self.longtermMem_fc = torch.zeros(batch_size,self.lm_len,self.hidden_size).cuda()
          self.embs = torch.empty((batch_size, self.seq_len, self.hidden_size)).cuda()
        else:
          out = o

        
        if h0 == None and c0 == None:
          h0 = torch.zeros(self.num_layers,batch_size,self.hidden_size).cuda()
          c0 = torch.zeros(self.num_layers,batch_size,self.hidden_size).cuda()

        hidden_state = h0
        cell_state = c0
        outputs = torch.empty((batch_size, self.seq_len, 2)).cuda()
        
        output = None

        for t in range(self.seq_len):
        
            # for the first time step the input is the feature vector
            if output == None:
              output = torch.zeros(batch_size,self.hidden_size).cuda()
              outesread1_lstm = torch.zeros(batch_size,self.hidden_size).cuda()
              outesreadl1_lstm = torch.zeros(batch_size,self.hidden_size).cuda()
              outesread1_fc = torch.zeros(batch_size,self.hidden_size).cuda()
              outesreadl1_fc = torch.zeros(batch_size,self.hidden_size).cuda()
            else:
              a1_lstm = self.alpha_lstm(output.view(batch_size,self.hidden_size))
              outesreada1_lstm = self.embs.clone()*a1_lstm.unsqueeze(2)#clone
              outesread1_lstm = torch.sum(outesreada1_lstm,dim=1)
              al1_lstm = self.alpha_long_lstm(output.view(batch_size,self.hidden_size))
              outesreada1_lstm = self.longtermMem_lstm.clone()*al1_lstm.unsqueeze(2)#clone
              outesreadl1_lstm = torch.sum(outesreada1_lstm,dim=1)
              

              
            combin_lstm = torch.cat([torch.flatten(features,start_dim=1),out,output.view(batch_size,self.hidden_size),outesread1_lstm,outesreadl1_lstm],dim=1)
            combin_lstm = self.inject_image_lstm(combin_lstm)
            
            self.embs[:, t, :] = output.view(batch_size,self.hidden_size).clone().detach()
            
            
            
            output, (hidden_state, cell_state) = self.lstm_cell(combin_lstm.unsqueeze(0), (hidden_state, cell_state))
            combin_lstm = None
            
            
            
            a1 = self.alpha_fc(output.view(batch_size,self.hidden_size))
            outesreada1 = self.embs.clone()*a1.unsqueeze(2)#clone
            outesread1 = torch.sum(outesreada1,dim=1)
            
            
            
            combin_fc = torch.cat([torch.flatten(features,start_dim=1),out,output.view(batch_size,self.hidden_size),outesread1_fc,outesreadl1_fc],dim=1)
            
            
            

            
            #combOut = torch.cat([combin_fc, output.view(batch_size,self.hidden_size),out,outesread1,outesreadl1],dim=1)#,outesread],dim=1)
            

            out = self.fc_out(combin_fc)
            out = out*32.0
            outputs[:, t, :] = out
            
            # build the output tensor
        self.longtermMem_lstm[:,ind,:] = output.view(batch_size,self.hidden_size).detach()
        #self.embs = self.embs.detach().clone()
        return outputs, hidden_state.detach(), cell_state.detach(), out.detach(),output.detach()#, captions


In [4]:
#torch.stack([torch.zeros(64,10),torch.ones(64,10)],dim = 0)[0,:,:]

In [5]:
a = torch.ones(64,256,50)
b = torch.from_numpy(np.random.randn(50))
c = a*b
print(c[0,1,:])
print(c[1,1,:])

tensor([-0.2207, -1.0992, -1.0747,  0.1374, -0.1018,  0.2264, -1.1673,  1.2917,
        -0.3174, -0.3125, -2.7375,  0.3416, -0.5759,  1.1575, -0.2408,  0.0979,
        -0.4704, -0.6451,  0.0421, -1.2225, -0.1202, -0.6983,  1.7923,  1.7498,
         1.6260,  0.8799, -1.8453,  0.1236, -1.2229,  0.7149, -0.5254, -0.7837,
         0.1936,  1.0847,  1.8004, -0.8017, -2.2928,  0.7022, -0.4231,  1.0132,
        -0.0291,  1.0558, -0.2044, -0.6308,  0.3832,  0.1892,  1.3569,  0.6463,
         1.3125, -0.9495], dtype=torch.float64)
tensor([-0.2207, -1.0992, -1.0747,  0.1374, -0.1018,  0.2264, -1.1673,  1.2917,
        -0.3174, -0.3125, -2.7375,  0.3416, -0.5759,  1.1575, -0.2408,  0.0979,
        -0.4704, -0.6451,  0.0421, -1.2225, -0.1202, -0.6983,  1.7923,  1.7498,
         1.6260,  0.8799, -1.8453,  0.1236, -1.2229,  0.7149, -0.5254, -0.7837,
         0.1936,  1.0847,  1.8004, -0.8017, -2.2928,  0.7022, -0.4231,  1.0132,
        -0.0291,  1.0558, -0.2044, -0.6308,  0.3832,  0.1892,  1.3569,  

In [6]:
model = DecoderRNN().cuda()

In [7]:
optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001, betas = (.9,.999))#ideal


In [8]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()


In [9]:
model = model.train()
for seql in range(3,24,4):
  #seql = 19
  for epoch in range (4):
    loss = None
    for x,y in loader_train:
      h0 = None
      c0 = None
      for i in range(seql):
        x = x.cuda()
        y = y.cuda()
        
        optimizer.zero_grad()
        
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
        loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        #writer.add_scalar("Loss/train", loss, epoch)
        
        loss.backward()
        optimizer.step()
      #print(y[:,0,:2])
      print('epoch',epoch,'seql',seql)
      print(loss)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 0 seql 3
tensor(37.2019, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(27.6436, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(15.9826, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(3.9513, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(4.2259, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(3.7908, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(3.2809, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(3.8927, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(4.4786, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(3.9438, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(3.6105, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(3.2696, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(2.5030, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 3
tensor(2.4918, device='cuda:0', grad_fn=<MeanB

In [10]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001, betas = (.9,.999))#ideal

for g in optimizer.param_groups:
    g['lr'] = 0.0001


In [11]:

model = model.train()
#for seql in range(15,20,4):
seql = 23
for epoch in range (15):
  loss = None
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 0 seql 23
tensor(1.3116, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.2901, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.5229, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.3082, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.6331, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.6529, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.7020, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.5718, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.5039, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.0923, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.2783, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.2903, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.5054, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.5163, device='cuda:0', gra

In [12]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.00001, betas = (.9,.999))#ideal
for g in optimizer.param_groups:
    g['lr'] = 0.00001



In [13]:

model = model.train()
#for seql in range(15,20,4):
seql = 23
for epoch in range (50):
  loss = None
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch 0 seql 23
tensor(9.8932, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(8.4587, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(5.6024, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(3.8047, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(3.2214, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(2.4198, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(2.1669, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.9565, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.4010, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.4805, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.3832, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.2603, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.1150, device='cuda:0', grad_fn=<MeanBackward0>)
epoch 0 seql 23
tensor(1.1425, device='cuda:0', gra

In [14]:

#optimizer.zero_grad()


In [15]:
"""model = model.train()
for epoch in range (20):
  loss = None
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(19):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      #print(x.shape,y.shape)
      if i == 0:
        out, h0, c0, o = model(x,y[:,i*50:50*(i+1),:2],h0,c0,ind = i)
      else:
        out, h0, c0, o = model(x,y[:,i*50:50*(i+1),:2],h0,c0,ind = i,o=o)
        
      loss = torch.mean((out-y[:,1+(50*i):1+(50*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)

      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch)
    print(loss)"""

'model = model.train()\nfor epoch in range (20):\n  loss = None\n  for x,y in loader_train:\n    h0 = None\n    c0 = None\n    for i in range(19):\n      x = x.cuda()\n      y = y.cuda()\n      \n      optimizer.zero_grad()\n      #print(x.shape,y.shape)\n      if i == 0:\n        out, h0, c0, o = model(x,y[:,i*50:50*(i+1),:2],h0,c0,ind = i)\n      else:\n        out, h0, c0, o = model(x,y[:,i*50:50*(i+1),:2],h0,c0,ind = i,o=o)\n        \n      loss = torch.mean((out-y[:,1+(50*i):1+(50*(i+1)),:2])**2)\n      writer.add_scalar("Loss/train", loss, epoch)\n\n      loss.backward()\n      optimizer.step()\n    #print(y[:,0,:2])\n    print(\'epoch\',epoch)\n    print(loss)'

In [16]:
from torch.utils import data
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

mini_batch = 100
test_dataset = DonutDataset(length = 100*10)
loader_test = data.DataLoader(
    test_dataset, 
    batch_size=mini_batch,
    sampler=RandomSampler(data_source=test_dataset),
    num_workers=2)


x0_y0_r_xrf_yrf torch.Size([1000, 1]) torch.Size([1000, 1]) torch.Size([1000, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


In [17]:
#model = model.eval()
with torch.no_grad():
  for epoch in range (1):
    loss = None
    for x,y in loader_test:
      h0 = None
      c0 = None
      seql = 23
      loss = torch.zeros(seql)
      for i in range(seql):
        optimizer.zero_grad()
        x = x.cuda()
        y = y.cuda()
        
        #print(x.shape,y.shape)
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
        loss[i] = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        print(loss[i])
        writer.add_scalar("Loss/val", loss[i],i, epoch)
        

      #print(y[:,0,:2])
      print('epoch',epoch)
      print('test loss', torch.mean(loss))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor(0.2493)
tensor(0.1879)
tensor(0.2490)
tensor(0.3118)
tensor(0.3558)
tensor(0.3878)
tensor(0.4778)
tensor(0.4982)
tensor(0.4096)
tensor(0.5135)
tensor(0.5573)
tensor(0.5903)
tensor(0.5598)
tensor(0.6408)
tensor(0.5034)
tensor(0.5933)
tensor(0.4990)
tensor(0.4550)
tensor(0.4280)
tensor(0.4197)
tensor(0.3377)
tensor(0.3259)
tensor(0.3034)
epoch 0
test loss tensor(0.4284)
tensor(0.2411)
tensor(0.1561)
tensor(0.2180)
tensor(0.2195)
tensor(0.3869)
tensor(0.4767)
tensor(0.4820)
tensor(0.4757)
tensor(0.6316)
tensor(0.5158)
tensor(0.7173)
tensor(0.7251)
tensor(0.7793)
tensor(0.7960)
tensor(0.5847)
tensor(0.8329)
tensor(0.5648)
tensor(0.5640)
tensor(0.5533)
tensor(0.5321)
tensor(0.4530)
tensor(0.3731)
tensor(0.3467)
epoch 0
test loss tensor(0.5055)
tensor(0.2350)
tensor(0.1436)
tensor(0.1995)
tensor(0.2005)
tensor(0.2434)
tensor(0.3512)
tensor(0.3292)
tensor(0.3317)
tensor(0.3659)
tensor(0.4178)
tensor(0.5129)
tensor(0.5285)
tensor(0.4782)
tensor(0.5313)
tensor(0.4638)
tensor(0.5142)
tens

In [18]:
writer.flush()
writer.close()


In [19]:
#%load_ext tensorboard

In [20]:
#!kill 3285
#%tensorboard --logdir runs

In [21]:
pred_dataset = DonutDataset(length = 100)

x0_y0_r_xrf_yrf torch.Size([100, 1]) torch.Size([100, 1]) torch.Size([100, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


In [22]:
!nvidia-smi

Wed May 26 22:45:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    50W / 250W |   1571MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [23]:
with torch.no_grad():
  model = model.eval()
  DonutDataset.displayCanvas('lstmpredictions.png',pred_dataset, model = model)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


open file:  lstmpredictions.png


In [24]:
with torch.no_grad():
  model = model.eval()
  DonutDataset.displayCanvas('trainlstmpredictions.png',dataset, model = model)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


open file:  trainlstmpredictions.png


In [25]:
for name, param in model.named_parameters():
  print(name)

lstm_cell.weight_ih_l0
lstm_cell.weight_hh_l0
lstm_cell.bias_ih_l0
lstm_cell.bias_hh_l0
lstm_cell.weight_ih_l1
lstm_cell.weight_hh_l1
lstm_cell.bias_ih_l1
lstm_cell.bias_hh_l1
inject_image_lstm.0.weight
inject_image_lstm.0.bias
fc_out.0.weight
fc_out.0.bias
alpha_lstm.0.weight
alpha_lstm.0.bias
alpha_fc.0.weight
alpha_fc.0.bias
alpha_long_lstm.0.weight
alpha_long_lstm.0.bias
